In [2]:
import numpy as np
import scipy.io as scio    
import os,re
from scipy import ndimage
from scipy import misc
from matplotlib import pyplot as plt

In [3]:
import keras

Using TensorFlow backend.


In [4]:
from keras.models import *
from keras.layers import Input,Flatten, Dense,merge, Conv2D, MaxPooling2D, UpSampling2D, Dropout, Cropping2D,Concatenate,Activation
from keras.optimizers import *
from keras.callbacks import ModelCheckpoint, LearningRateScheduler,ReduceLROnPlateau,CSVLogger
from keras import backend as keras

In [5]:
def atoi(text) : 
    return int(text) if text.isdigit() else text

In [6]:
def natural_keys(text) :
    return [atoi(c) for c in re.split('(\d+)', text)]

In [7]:
total_images = np.load('total_images.npy')
ground_truth_images = np.load('gt_labels_binary.npy')

In [12]:
root_path = "/home/iplab/AI_assignment/cancer-prediction/"

In [13]:
def get_filenames(path):
    filenames = []
    for root, dirnames, filenames in os.walk(path):
        filenames.sort(key = natural_keys)
        rootpath = root
    print(len(filenames))
    return filenames

In [14]:
filenames_melanoma = get_filenames(root_path+"melanoma_resized/")
filenames_others = get_filenames(root_path+"others_resized/")

374
1626


In [15]:
filenames_total = filenames_melanoma+filenames_others
filenames_total.sort(key = natural_keys)

In [17]:
segmented_images = np.copy(total_images)
x,y,z = segmented_images[0].shape
for i in range(len(total_images)):
    for j in range(x):
        for k in range(y):
            for l in range(z):
                segmented_images[i][j][k][l] = total_images[i][j][k][l] if ground_truth_images[i][j][k]==1 else 0
    misc.imsave(root_path+"segmented_images/segmented_"+filenames_total[i],segmented_images[i])

In [18]:
segmented_images[0].shape

(128, 192, 3)

In [19]:
total_images.shape

(2000, 128, 192, 3)

In [20]:
ground_truth_images.shape

(2000, 128, 192)

In [21]:
np.save('segmented_images.npy',segmented_images)

In [22]:
classification_labels = np.zeros((len(total_images)))
i = 0
for file in filenames_total:
    if os.path.exists(root_path+"melanoma_resized/"+file):
        classification_labels[i]=0
    else:
        classification_labels[i]=1
    i+=1

In [23]:
print(classification_labels)

[ 1.  1.  0. ...,  1.  0.  1.]


In [24]:
np.save('classification_labels.npy',classification_labels)

In [25]:
train_indices = np.random.choice(2000,1900,replace=False)
print(len(train_indices))

1900


In [26]:
train_images = [segmented_images[i] for i in train_indices]
train_labels = [classification_labels[i] for i in train_indices]

In [27]:
print(len(train_images))
print(len(train_labels))

1900
1900


In [28]:
test_indices = [i for i in range(2000) if i not in train_indices]
print(len(test_indices))

100


In [29]:
test_images = [segmented_images[i] for i in test_indices]
test_labels = [classification_labels[i] for i in test_indices]

In [30]:
print(len(test_images))
print(len(test_labels))

100
100


In [31]:
plt.imshow(segmented_images[0])
plt.show()

In [32]:
print(classification_labels[0])

1.0


In [33]:
image_dims = segmented_images[0].shape
print(image_dims)

(128, 192, 3)


In [34]:
train_mean = np.mean(train_images,axis = (0,1,2,3))
train_std = np.std(train_images,axis = (0,1,2,3))
train_images = (train_images - train_mean)/(train_std+1e-7)

In [35]:
test_mean = np.mean(test_images,axis=(0,1,2,3))
test_std = np.std(test_images,axis = (0,1,2,3))
test_images = (test_images-test_mean)/(test_std+1e-7)

In [36]:
np.save('classification_train_images.npy',train_images)
np.save('classification_test_images.npy',test_images)
np.save('classification_train_labels.npy',train_labels)
np.save('classification_test_labels.npy',test_labels)

In [37]:
train_images = np.load('classification_train_images.npy')
test_images = np.load('classification_test_images.npy')
train_labels = np.load('classification_train_labels.npy')
test_labels =  np.load('classification_test_labels.npy')

In [38]:
def get_unet_model(image_dims):
    inputs = Input((image_dims[0],image_dims[1],image_dims[2]))
    conv1 = Conv2D(16, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(inputs)
    conv1 = Conv2D(16, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv1)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)

    conv2 = Conv2D(32, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool1)
    conv2 = Conv2D(32, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv2)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)

    conv3 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool2)
    conv3 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv3)
    pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)

    conv4 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool3)
    conv4 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv4)
    pool4 = MaxPooling2D(pool_size=(2, 2))(conv4)

    conv5 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool4)
    conv5 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv5)
   

    up6 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv5))
    merge6 = Concatenate(axis=3)([conv4,up6])
    conv6 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge6)
    conv6 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv6)

    up7 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv6))
    merge7 = Concatenate(axis=3)([conv3,up7])
    conv7 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge7)
    conv7 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv7)

    up8 = Conv2D(32, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv7))
    merge8 = Concatenate(axis=3)([conv2,up8])
    conv8 = Conv2D(32, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge8)
    conv8 = Conv2D(32, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv8)

    up9 = Conv2D(16, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv8))
    merge9 = Concatenate(axis=3)([conv1,up9])
    conv9 = Conv2D(16, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge9)
    conv9 = Conv2D(16, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv9)
  #  conv9 = Conv2D(2, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv9)
    #conv10 = Conv2D(1, 1, activation = 'sigmoid')(conv9)
    flatten1 = Flatten()(conv9)
    dense1 = Dense(256)(flatten1)
    dense2 = Dense(2)(dense1)
    output = Activation('softmax')(dense2)
    
    model = Model(inputs = inputs, outputs = output)
    return model

In [39]:
model = get_unet_model((128,192,3))

In [40]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 128, 192, 3)       0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 128, 192, 16)      448       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 128, 192, 16)      2320      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 64, 96, 16)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 64, 96, 32)        4640      
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 64, 96, 32)        9248      
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 32, 48, 32)        0         
__________

In [41]:
smooth = 1

In [42]:
def dice_coef(y_true, y_pred):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    
    return (2. * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)


In [43]:
def dice_coef_loss(y_true, y_pred):
    return -dice_coef(y_true, y_pred)

In [44]:
model.compile(optimizer = Adam(lr = 1e-25), loss = dice_coef_loss, metrics = ['accuracy'])

In [45]:
lr_reducer = ReduceLROnPlateau(factor=0.5, cooldown=0, patience=6, min_lr=0.5e-6)
csv_logger = CSVLogger('Unet_lr_e4_bs_10_classifier.csv')
model_checkpoint = ModelCheckpoint("Unet_lr_e4_bs_10_classifier.hdf5",monitor = 'val_loss',verbose = 1,save_best_only=True)

In [46]:
model.fit(train_images, train_labels, batch_size=10, epochs=20, verbose=1,validation_data=(test_images,test_labels), shuffle=True, callbacks=[lr_reducer,csv_logger,model_checkpoint])

Train on 1900 samples, validate on 100 samples
Epoch 1/20


InternalError: Dst tensor is not initialized.
	 [[Node: Const_101 = Const[dtype=DT_FLOAT, value=Tensor<type: float shape: [393216,256] values: [0 0 0]...>, _device="/job:localhost/replica:0/task:0/gpu:0"]()]]

Caused by op u'Const_101', defined at:
  File "/home/iplab/anaconda2/lib/python2.7/runpy.py", line 174, in _run_module_as_main
    "__main__", fname, loader, pkg_name)
  File "/home/iplab/anaconda2/lib/python2.7/runpy.py", line 72, in _run_code
    exec code in run_globals
  File "/home/iplab/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py", line 3, in <module>
    app.launch_new_instance()
  File "/home/iplab/anaconda2/lib/python2.7/site-packages/traitlets/config/application.py", line 596, in launch_instance
    app.start()
  File "/home/iplab/anaconda2/lib/python2.7/site-packages/ipykernel/kernelapp.py", line 442, in start
    ioloop.IOLoop.instance().start()
  File "/home/iplab/anaconda2/lib/python2.7/site-packages/zmq/eventloop/ioloop.py", line 162, in start
    s